<a href="https://colab.research.google.com/github/flyaflya/persuasive/blob/main/notebookDemos/HomeworkReviewSessionDataManip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Assign
2. Subset
3. Drop
4. Sort
5. Aggregate
6. Split-Apply-Combine
7. Merge

In [1]:
import pandas as pd
import xarray as xr

## get primarys cast in the primaries as of February 25th, 2016. 
primaryDF = pd.read_csv("https://raw.githubusercontent.com/flyaflya/persuasive/main/primary_results.csv")

## get county demographic information from 2016
countyDF = pd.read_csv("https://raw.githubusercontent.com/flyaflya/persuasive/main/county_facts.csv")

# Since each observation in dataframe is a unique countyID, this data should be indexed by countyID.
# https://docs.xarray.dev/en/stable/user-guide/pandas.html#working-with-pandas
countyDS = (
    countyDF
    .set_index("countyID") ## this index will be dimension of new dataset
    # chain this newly indexed dataframe to a function using the pipe() method
    .pipe(xr.Dataset.from_dataframe) ## pipe is panda function that passes df
                                     ## as first argument to the xarray function
                                     ## which creates a dataset from a df                                 
)

# Data should be indexed by unique identifiers.  In this case,
# need combination of countyID and candidate
primaryDS = (
    primaryDF
    .set_index(["countyID","candidate"]) ## this index will be dimension of new dataset
    # chain this newly indexed dataframe to a function using the pipe() method
    .pipe(xr.Dataset.from_dataframe) ## pipe is panda function that passes df
                                     ## as first argument to the xarray function
                                     ## which creates a dataset from a df                                 
)
# merge datasets and drop na's
mergedDS = (
    primaryDS
    .merge(countyDS, join = 'left')
)
mergedDS

<xarray.Dataset>
Dimensions:                (countyID: 171, candidate: 15)
Coordinates:
  * countyID               (countyID) int64 19001 19003 19005 ... 45089 45091
  * candidate              (candidate) object ' Uncommitted' ... 'Ted Cruz'
Data variables:
    party                  (countyID, candidate) object 'Democrat' ... 'Repub...
    votes                  (countyID, candidate) float64 0.0 61.0 ... 9.169e+03
    areaName               (countyID) object 'Adair County' ... 'York County'
    stateAbbrev            (countyID) object 'IA' 'IA' 'IA' ... 'SC' 'SC' 'SC'
    population2014         (countyID) int64 7454 3875 14038 ... 32695 245346
    medianHouseholdIncome  (countyID) int64 47892 45871 48831 ... 25849 53740
    populationPerSqMile    (countyID) float64 13.5 9.5 22.4 ... 56.3 36.8 332.2

Which county (areaName) had the largest number of voters for Republican candidates?   --- Adair County 

In [50]:
(
    mergedDS
    ## subset to just republican candidates
    .where(mergedDS.party == "Republican", drop = True)
    .assign(totalVotes = lambda DS: DS.votes.sum(dim = "candidate"))
    .assign(pctVotesInCounty = lambda DS: DS.votes / DS.totalVotes)
).to_dataframe().sort_values("totalVotes", ascending = False)

party    votes           areaName stateAbbrev  \
countyID candidate                                                           
45045    Carly Fiorina         NaN      NaN                NaN         NaN   
         Marco Rubio    Republican  22875.0  Greenville County          SC   
         Ted Cruz       Republican  22899.0  Greenville County          SC   
         Rick Santorum         NaN      NaN                NaN         NaN   
         Rand Paul             NaN      NaN                NaN         NaN   
...                            ...      ...                ...         ...   
32009    Jeb Bush              NaN      NaN                NaN         NaN   
         Marco Rubio    Republican      8.0   Esmeralda County          NV   
         Mike Huckabee         NaN      NaN                NaN         NaN   
         Rand Paul             NaN      NaN                NaN         NaN   
         John Kasich    Republican      3.0   Esmeralda County          NV   

                        population2014  medianHouseholdIncome  \
countyID candidate                                              
45045    Carly Fiorina             NaN                    NaN   
         Marco Rubio          482752.0                49022.0   
         Ted Cruz             482752.0                49022.0   
         Rick Santorum             NaN                    NaN   
         Rand Paul                 NaN                    NaN   
...                                ...                    ...   
32009    Jeb Bush                  NaN                    NaN   
         Marco Rubio             822.0                30284.0   
         Mike Huckabee             NaN                    NaN   
         Rand Paul                 NaN                    NaN   
         John Kasich             822.0                30284.0   

                        populationPerSqMile  totalVotes  pctVotesInCounty  
countyID candidate                                                         
45045    Carly Fiorina                  NaN     92954.0               NaN  
         Marco Rubio                  574.7     92954.0          0.246089  
         Ted Cruz                     574.7     92954.0          0.246348  
         Rick Santorum                  NaN     92954.0               NaN  
         Rand Paul                      NaN     92954.0               NaN  
...                                     ...         ...               ...  
32009    Jeb Bush                       NaN        73.0               NaN  
         Marco Rubio                    0.2        73.0          0.109589  
         Mike Huckabee                  NaN        73.0               NaN  
         Rand Paul                      NaN        73.0               NaN  
         John Kasich                    0.2        73.0          0.041096  

[1881 rows x 9 columns]

debuggin Sara's code

In [46]:
# Grouping Republicans by Areaname 
RepublicanDS=(
    mergedDS
    .where(mergedDS.party =="Republican")
    .get(["areaName","stateAbbrev","party","votes"])
    # .groupby("areaName")
    # .sum(...)
    # .sortby("votes",ascending = False)
)
RepublicanDS.to_dataframe().dropna()
# RepublicanDS

areaName stateAbbrev       party    votes
countyID candidate                                                    
19001    Ben Carson      Adair County          IA  Republican     61.0
         Carly Fiorina   Adair County          IA  Republican     16.0
         Chris Christie  Adair County          IA  Republican      4.0
         Donald Trump    Adair County          IA  Republican    104.0
         Jeb Bush        Adair County          IA  Republican      4.0
...                               ...         ...         ...      ...
45091    Donald Trump     York County          SC  Republican  12031.0
         Jeb Bush         York County          SC  Republican   2036.0
         John Kasich      York County          SC  Republican   2632.0
         Marco Rubio      York County          SC  Republican   8089.0
         Ted Cruz         York County          SC  Republican   9169.0

[1519 rows x 4 columns]

## Leading Republican Candidate has 419,951 votes
Total Votes for each republican candidate

In [59]:
(
    mergedDS
    ## subset to just republican candidates
    .where(mergedDS.party == "Republican", drop = True)
    .assign(totalVotes = lambda DS: DS.votes.sum(dim = "countyID"))
    .get("totalVotes")
).to_dataframe()

,totalVotes
candidate,
Ben Carson,80627.0
Carly Fiorina,15181.0
Chris Christie,24347.0
Donald Trump,419951.0
Jeb Bush,94394.0
John Kasich,107290.0
Marco Rubio,256736.0
Mike Huckabee,3300.0
Rand Paul,8460.0


### Which state (stateAbbrev) had the county (areaName) where 22,800 voters turned out for the Democrat primary?  - Polk County	... ANSWER: IA

In [73]:
(
    mergedDS
    .where(mergedDS.party == "Democrat", drop = True)
    .assign(totalVotes = lambda DS: DS.votes.sum(dim = "candidate"))
    .where(lambda DS: DS.totalVotes == 22800, drop = True)
).to_dataframe().sort_values("totalVotes", ascending = False)

party    votes     areaName stateAbbrev  \
countyID candidate                                                     
19153     Uncommitted     Democrat      0.0  Polk County          IA   
         Bernie Sanders   Democrat  10526.0  Polk County          IA   
         Hillary Clinton  Democrat  12122.0  Polk County          IA   
         Martin O'Malley  Democrat    152.0  Polk County          IA   

                          population2014  medianHouseholdIncome  \
countyID candidate                                                
19153     Uncommitted           459862.0                59018.0   
         Bernie Sanders         459862.0                59018.0   
         Hillary Clinton        459862.0                59018.0   
         Martin O'Malley        459862.0                59018.0   

                          populationPerSqMile  totalVotes  
countyID candidate                                         
19153     Uncommitted                   750.5     22800.0  
         Bernie Sanders                 750.5     22800.0  
         Hillary Clinton                750.5     22800.0  
         Martin O'Malley                750.5     22800.0

### What is the wealthiest county in Nebraska (NE)?  (enter text exactly as it appears in the data, for example "New Castle County") this state did not have a primary yet   ANSWR: Sarpy County	

In [79]:
(  #method 1
    countyDS
    .where(countyDS.stateAbbrev == "NE", drop = True)
    .sortby("medianHouseholdIncome", ascending = False)
).to_dataframe()

,areaName,stateAbbrev,population2014,medianHouseholdIncome,populationPerSqMile
countyID,,,,,
31153,Sarpy County,NE,172193.0,69965.0,664.6
31177,Washington County,NE,20258.0,65409.0,51.9
31025,Cass County,NE,25524.0,64697.0,45.3
31155,Saunders County,NE,20919.0,59870.0,27.7
31159,Seward County,NE,17150.0,59435.0,29.3
...,...,...,...,...,...
31129,Nuckolls County,NE,4369.0,37594.0,7.8
31133,Pawnee County,NE,2702.0,37321.0,6.4
31103,Keya Paha County,NE,810.0,36923.0,1.1


In [81]:
(  #method 2
    countyDS
    .where(countyDS.stateAbbrev == "NE", drop = True)
    .where(lambda DS: DS.medianHouseholdIncome == max(DS.medianHouseholdIncome), drop = True)
).to_dataframe()

,areaName,stateAbbrev,population2014,medianHouseholdIncome,populationPerSqMile
countyID,,,,,
31153,Sarpy County,NE,172193.0,69965.0,664.6


### How many votes has the leading Democrat presidential candidate accumulated? -- 226,536

In [83]:
(
    mergedDS
    ## subset to just democrat candidates
    .where(mergedDS.party == "Democrat", drop = True)
    .assign(totalVotes = lambda DS: DS.votes.sum(dim = "countyID"))
    .get("totalVotes")
).to_dataframe()

,totalVotes
candidate,
Uncommitted,43.0
Bernie Sanders,226536.0
Hillary Clinton,171129.0
Martin O'Malley,747.0


In [90]:
## pandas way
(
    primaryDF
    .groupby("candidate")
    .agg(totalVotes=('votes','sum'))
)



,totalVotes
candidate,
Uncommitted,43
Ben Carson,80627
Bernie Sanders,226536
Carly Fiorina,15181
Chris Christie,24347
Donald Trump,419951
Hillary Clinton,171129
Jeb Bush,94394
John Kasich,107290


### For this question, assume that there is no distinction between Republicans and Democrats; all votes cast count equally.  Calculate the percentage of all votes (Democrat + Republican) placed in each county that each candidate received.  For the county where Hillary Clinton achieved her largest percentage of votes, what percentage of the vote did she earn (enter as decimal to the nearest thousandth place, for example, 0.062, 0.124, etc)?  ANSWER: 0.308

In [110]:
( #Calculate the percentage of all votes (Democrat + Republican) placed in each county that each candidate received
    mergedDS
    .assign(totalVotesByCounty = lambda DS: DS.votes.sum(dim="candidate"))
    .sel(candidate = "Hillary Clinton")
    .assign(pctOfCountyVote = lambda DS: DS.votes / DS.totalVotesByCounty)
    .where(lambda DS: DS.votes.notnull(), drop = True)
).to_dataframe().sort_values("pctOfCountyVote", ascending = False)

,party,votes,areaName,stateAbbrev,population2014,medianHouseholdIncome,populationPerSqMile,totalVotesByCounty,pctOfCountyVote,candidate
countyID,,,,,,,,,,
19111,Democrat,929.0,Lee County,IA,35286.0,42469.0,69.3,3014.0,0.308228,Hillary Clinton
19037,Democrat,360.0,Chickasaw County,IA,12264.0,43971.0,24.7,1199.0,0.300250,Hillary Clinton
19131,Democrat,260.0,Mitchell County,IA,10779.0,51078.0,23.0,871.0,0.298507,Hillary Clinton
19067,Democrat,453.0,Floyd County,IA,16077.0,43826.0,32.6,1559.0,0.290571,Hillary Clinton
19019,Democrat,525.0,Buchanan County,IA,21038.0,55553.0,36.7,1837.0,0.285792,Hillary Clinton
...,...,...,...,...,...,...,...,...,...,...
32013,Democrat,27.0,Humboldt County,NV,17279.0,59472.0,1.7,939.0,0.028754,Hillary Clinton
32019,Democrat,80.0,Lyon County,NV,51789.0,46137.0,26.0,3059.0,0.026152,Hillary Clinton
32005,Democrat,85.0,Douglas County,NV,47536.0,60100.0,66.2,4766.0,0.017835,Hillary Clinton


In [115]:
( #Calculate the percentage of all votes (Democrat + Republican) placed in each county that each candidate received
    mergedDS
    .assign(totalVotesByCounty = lambda DS: DS.votes.sum(dim="candidate"))
    .sel(candidate = "Hillary Clinton")
    .assign(pctOfCountyVote = lambda DS: DS.votes / DS.totalVotesByCounty)
    .where(lambda DS: DS.votes, drop = True)
    .dropna(dim = "countyID")
).to_dataframe().sort_values("pctOfCountyVote", ascending = False)

,party,votes,areaName,stateAbbrev,population2014,medianHouseholdIncome,populationPerSqMile,totalVotesByCounty,pctOfCountyVote,candidate
countyID,,,,,,,,,,
19111,Democrat,929.0,Lee County,IA,35286.0,42469.0,69.3,3014.0,0.308228,Hillary Clinton
19037,Democrat,360.0,Chickasaw County,IA,12264.0,43971.0,24.7,1199.0,0.300250,Hillary Clinton
19131,Democrat,260.0,Mitchell County,IA,10779.0,51078.0,23.0,871.0,0.298507,Hillary Clinton
19067,Democrat,453.0,Floyd County,IA,16077.0,43826.0,32.6,1559.0,0.290571,Hillary Clinton
19019,Democrat,525.0,Buchanan County,IA,21038.0,55553.0,36.7,1837.0,0.285792,Hillary Clinton
...,...,...,...,...,...,...,...,...,...,...
32013,Democrat,27.0,Humboldt County,NV,17279.0,59472.0,1.7,939.0,0.028754,Hillary Clinton
32019,Democrat,80.0,Lyon County,NV,51789.0,46137.0,26.0,3059.0,0.026152,Hillary Clinton
32005,Democrat,85.0,Douglas County,NV,47536.0,60100.0,66.2,4766.0,0.017835,Hillary Clinton


### What percentage of the republican vote in Iowa did Rand Paul receive among households with a median income greater than $50,000 (enter as decimal to the nearest thousandth place, for example, 0.062, 0.124, etc)?

In [125]:
(
    mergedDS
    .where(lambda DS: DS.medianHouseholdIncome > 50000, drop = True)
    .where(lambda DS: DS.party == "Republican", drop = True)
    .where(lambda DS: DS.stateAbbrev == "IA", drop = True)
    .sum(dim = "countyID")
    .assign(totalVoteInIA = lambda DS: DS.votes.sum(...))
    .assign(pctOfStateVote = lambda DS: DS.votes / DS.totalVoteInIA)
).to_dataframe()

,votes,population2014,medianHouseholdIncome,populationPerSqMile,totalVoteInIA,pctOfStateVote
candidate,,,,,,
Ben Carson,11585.0,2037562.0,2244181.0,3336.5,129513.0,0.089450
Carly Fiorina,2365.0,2037562.0,2244181.0,3336.5,129513.0,0.018261
Chris Christie,2460.0,2037562.0,2244181.0,3336.5,129513.0,0.018994
Donald Trump,29548.0,2037562.0,2244181.0,3336.5,129513.0,0.228147
Jeb Bush,3831.0,2037562.0,2244181.0,3336.5,129513.0,0.029580
John Kasich,2872.0,2037562.0,2244181.0,3336.5,129513.0,0.022175
Marco Rubio,32225.0,2037562.0,2244181.0,3336.5,129513.0,0.248817
Mike Huckabee,2005.0,2037562.0,2244181.0,3336.5,129513.0,0.015481
Rand Paul,6583.0,2037562.0,2244181.0,3336.5,129513.0,0.050829
